In [1]:
import os
import multiprocessing
import time
import numpy as np
from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit, Parameter
from qiskit import Aer, execute
from numpy.random import default_rng
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
def add_gate_unit(qc, angles, qubits, barrier=False):
    qc.cx(qubits[0], qubits[1])
    qc.cx(qubits[1], qubits[0])
    qc.cu(angles[0], angles[1], angles[2], 0., qubits[1], qubits[0])
    qc.cx(qubits[1], qubits[0])
    qc.p(angles[3], qubits[1])
    qc.cx(qubits[0], qubits[1])
    qc.p(angles[4], qubits[0])
    qc.p(angles[4], qubits[1])
    if barrier:
        qc.barrier()

In [3]:
rng = default_rng()

circuit = QuantumCircuit(4)
angles = np.pi * rng.random(3)
angles[1:] *= 2.
circuit.x(0)
circuit.x(2)
add_zerou(circuit, angles, [1, 3])
backend = Aer.get_backend('statevector_simulator')
backend.set_options(method='statevector_gpu')

job = execute(circuit, backend)

data = job.result().data()
print(data['statevector'])

In [4]:
MAX_SIZE = 30

@tf.function
def compute_purity_tf(statevector_real, statevector_imag):
    prod_real = tf.tensordot(statevector_real, statevector_real, axes=[[1], [1]])
    prod_real += tf.tensordot(statevector_imag, statevector_imag, axes=[[1], [1]])
    prod_imag = tf.tensordot(statevector_real, statevector_imag, axes=[[1], [1]])
    prod_imag -= tf.tensordot(statevector_imag, statevector_real, axes=[[1], [1]])
    purity = tf.math.reduce_sum(tf.math.square(prod_real))
    purity += tf.math.reduce_sum(tf.math.square(prod_imag))
    
    return purity

def get_renyi_entropy_tf(statevector, subsystem):
    system_size = int(np.log2(statevector.shape[0]))
    subsystem_size = len(subsystem)

    if system_size <= MAX_SIZE:
        low = min(subsystem)
        high = system_size - max(subsystem) - 1

        slices = [np.s_[:]]
    elif max(subsystem) < MAX_SIZE:
        low = min(subsystem)
        high = MAX_SIZE - max(subsystem) - 1

        block_size = 2 ** MAX_SIZE
        slices = [np.s_[block_size * i:block_size * (i + 1)] for i in range(2 ** (system_size - MAX_SIZE))]
    else:
        raise RuntimeError('don\'t do this it\'s too much work')
        
    shape = (2 ** high, 2 ** subsystem_size, 2 ** low)
    tfshape = (2 ** subsystem_size, 2 ** (high + low))
        
    purity = 0.
    for sl in slices:
        statevector = np.reshape(statevector[sl], shape).transpose((1, 0, 2)) # another reshape will cause data copy
        real = tf.constant(statevector.real, shape=tfshape)
        imag = tf.constant(statevector.imag, shape=tfshape)
        purity += compute_purity_tf(real, imag).numpy()
    entropy = -np.log2(purity)

    return entropy / subsystem_size

In [5]:
def get_statevectors(circuit, get_last=False):
    backend = Aer.get_backend('statevector_simulator')
    backend.set_options(method='statevector_gpu')

    start = time.time()
    job = execute(circuit, backend)
    data = job.result().data()
    print('{} seconds to execute the circuit'.format(time.time() - start))
    
    steps = sorted(int(key.replace('step', '')) for key in data['snapshots']['statevector'].keys())
    statevectors = list(data['snapshots']['statevector']['step{}'.format(istep)][0] for istep in steps)
    if get_last:
        statevectors.append(data['statevector'])
        
    return statevectors

def compute_entropies(statevectors, subsystem):
    entropies = np.empty(len(statevectors), dtype=np.float)
    for isample, statevector in enumerate(statevectors):
        start = time.time()
        entropies[isample] = get_renyi_entropy_tf(statevector, subsystem)
        print('{} seconds to compute entropy for sample {}'.format(time.time() - start, isample))

    return entropies

In [6]:
def initialize_circuit(system_size):
    circuit = QuantumCircuit(system_size)
    
    for iq in range(0, system_size, 2):
        circuit.x(iq)
        
    return circuit

In [7]:
num_experiments = 1
sequence_length = 99
sample_every = 3

system_size = 30
subsystem = range(11, 19) # [0-10] [11-18] [19-29]

total_ram_in_gb = 350
max_snapshots = int((2 ** 30) * total_ram_in_gb // (2 ** (4 + system_size) * 1.2)) #1.2 : safety factor
print(max_snapshots)

18


In [ ]:
barrier = False

entropies = np.empty((num_experiments, sequence_length // sample_every + 1), dtype=np.float)
for iexp in range(num_experiments):
    entropies[iexp, 0] = 0.

    qubits_record = np.empty((sequence_length, 2), dtype=np.int)
    angles_record = np.empty((sequence_length, 5), dtype=np.float)
    num_snapshots = 0
    num_entropies = 1
    
    circuit = initialize_circuit(system_size)
        
    for istep in range(sequence_length):
        qubits = rng.integers(0, system_size, 2)
        while qubits[0] == qubits[1]:
            qubits = rng.integers(0, system_size, 2)

        angles = np.pi * rng.random(5)
        angles[1:] *= 2.
        add_gate_unit(circuit, angles, qubits, barrier=barrier)

        qubits_record[istep] = qubits
        angles_record[istep] = angles
        
        if (istep + 1) % sample_every == 0:
            circuit.snapshot('step{}'.format(istep))
            num_snapshots += 1
            
        if num_snapshots == max_snapshots:
            print('reached max snapshots, evaluating the entropies so far')
            statevectors = get_statevectors(circuit)
            entropies[iexp, num_entropies:num_entropies + num_snapshots] = compute_entropies(statevectors, subsystem)
            
            print('initializing and fast-forwarding the circuit')
            circuit = initialize_circuit(system_size)
            for ifwd in range(istep + 1):
                add_gate_unit(circuit, angles_record[ifwd], qubits_record[ifwd], barrier=barrier)
                
            num_entropies += num_snapshots
            num_snapshots = 0
            
    if num_entropies != entropies.shape[1]:
        get_last = ((istep + 1) % sample_every != 0)
        print('evaluating the last entropies')
        statevectors = get_statevectors(circuit, get_last=get_last)
        entropies[iexp, num_entropies:] = compute_entropies(statevectors, subsystem)

reached max snapshots, evaluating the entropies so far
178.38456773757935 seconds to execute the circuit
22.639429807662964 seconds to compute entropy for sample 0
20.974830865859985 seconds to compute entropy for sample 1
21.005354166030884 seconds to compute entropy for sample 2
20.990955114364624 seconds to compute entropy for sample 3
20.949779510498047 seconds to compute entropy for sample 4
20.97826862335205 seconds to compute entropy for sample 5
20.992107629776 seconds to compute entropy for sample 6
20.873960494995117 seconds to compute entropy for sample 7
20.887410879135132 seconds to compute entropy for sample 8
20.923527002334595 seconds to compute entropy for sample 9
20.887908697128296 seconds to compute entropy for sample 10
20.885396718978882 seconds to compute entropy for sample 11
20.885669231414795 seconds to compute entropy for sample 12
20.918928384780884 seconds to compute entropy for sample 13
20.920972108840942 seconds to compute entropy for sample 14
20.837542

In [ ]:
plt.plot(np.linspace(0., sequence_length, entropies.shape[1]), entropies[0])